## Ingestão de dados de API com token (Python) - Camada Bronze

In [0]:
import requests
from pyspark.sql import SparkSession
import os
from datetime import datetime, timedelta
from pyspark.sql import Row

In [0]:
spark = SparkSession.builder.getOrCreate()
import os

os.environ["freecryptofree_api_key"] = "1234567"

In [0]:
api_key = os.getenv("freecryptofree_api_key")
base_url = "https://api.freecryptoapi.com/v1"

headers = {
    "accept": "*/*",
    "Authorization": f"Bearer {api_key}"
}

cryptos = {
    "BTC",
    "ETC",
    "SOL"
}

##### Função para capturar os dados

In [0]:
def get_crypto(symbol: str):

    params = {
        "symbol": symbol
    }

    response = requests.get(
        f"{base_url}/getData",
        headers=headers,
        params=params,
        timeout=30
    )

    response.raise_for_status()
    return response.json()

###### Ingestão dos Dados Bronze

In [0]:
def ingest_bronze(symbol: str):
    execution_date = datetime.utcnow().date()
    rows = []

    response = get_crypto(symbol)

    for item in response["symbols"]:
        rows.append(Row(
            symbol=item["symbol"],
            last=item["last"],
            lowest=item["lowest"],
            highest=item["highest"],
            daily_change_percentage=item["daily_change_percentage"],
            source_exchange=item["source_exchange"],
            api_datetime=item["date"],
            ingestion_date=str(execution_date)
        ))
        
    df_bronze = spark.createDataFrame(rows)

    (
        df_bronze.write
        .format("delta")
        .mode("append")
        .saveAsTable("workspace.bronze.bronze_crypto_prices")
    )

###### Fazendo a chamada do metodo para ingestão

In [0]:
for crypto in list(cryptos):
    ingest_bronze(crypto)

# 